In [1]:
import pandas as pd 
import numpy as np
from docx.api import Document
import re
import json

# templates = [["header",1,"Metro is making more service changes.","Metro está haciendo más cambios en sus servicios.","Metro正在進行更多服務調整。","Metro hiện đang thực hiện nhiều thay đổi về dịch vụ.","메트로 서비스가 더욱 새롭게단장하고 있습니다.","メトロのサービスが変更されます。","Metro-ն կրկին փոփոխություններ է իրականացնում ծառայությունների մեջ:","Metro вносит дополнительные изменения в схемы движения."]]
# templates = ["header",1],["summary",1],["details",1],["end",1]
# final_template = pd.DataFrame(templates,columns=["section","order","en","es","zh-TW","vi","ko","ja","hy","ru"])


In [2]:
document = Document('data/202109shakeup.docx')
table = document.tables[0]
data = [[cell.text.replace("\n"," ").replace('"','').replace('" ','').lstrip() for cell in row.cells] for row in table.rows]

df = pd.DataFrame(data)
new_header = df.iloc[0]
df = df[1:] 
df.columns = new_header
# print(df.columns)
df = df.rename(columns={'English':'en','Spanish':'es','Chinese (Traditional)':'zh-TW','Korean':'ko','Vietnamese':'vi','Japanese':'ja','Russian':'ru','Armenian':'hy'})
# df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)

df = df.replace(' +',r' ',regex=True)
df = df.replace('"',r'',regex=True)
# df.to_json('test.json')
# df.to_csv('test.csv')
df.head()

final_df = pd.DataFrame(columns=["section","order","line","altline","en","es","zh-TW","vi","ko","ja","hy","ru","schedule"])

## Populating the data

### 2.1 Adding the `Summary` sections

In [3]:
def reset_final_df():
    return pd.DataFrame(columns=["section","order","line","altline","en","es","zh-TW","vi","ko","ja","hy","ru","schedule"])

In [5]:
header1 = df.loc[(df['en'].str.contains('\u2013') == False) & (df['en'].str.contains('Metro is making service'))]
header1 = header1.assign(section='header')
header1 = header1.assign(order='1')

header2 = df.loc[(df['en'].str.contains('\u2013') == False) & (df['en'].str.contains('New schedules start'))]
header2 = header2.assign(section='header')
header2 = header2.assign(order='2')

if not final_df.empty:
    final_df = reset_final_df()

final_df = final_df.append(header1)
final_df = final_df.append(header2)

final_df

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,schedule
34,header,1,NaN,NaN,Metro is making service changes.,Metro está haciendo cambios en el servicio.,Metro將對服務進行變更。,Metro sắp tiến hành thay đổi dịch vụ.,Metro는 버스편 운행 변경을 진행하고 있습니다.,Metroはサービスを変更しています。,Metro պոլիտենը փոփոխություններ է կատարում ծառա...,Metro вносит ряд изменений в расписание движения.,NaN
35,header,2,NaN,NaN,"New schedules start September 12, 2021.",Los nuevos horarios comienzan el 12 de septiem...,新安排將從2021年9月12日開始。,Lịch hoạt động mới sẽ bắt đầu có hiệu lực vào ...,신설 운행 일정은 2021년 9월 12일부터 시작합니다.,新しいスケジュールは2021年9月12日から開始されます。,Նոր ժամանակացույցերը կմեկնարկեն 2021 թվականի ս...,Новое расписание вводится в действие 12 сентяб...,NaN


In [4]:
final_df

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,schedule


In [6]:
# th = df[df['en'].str.contains('Starting on'):df['en'].str.contains('We’re ')]
th = df.loc[(df['en'].str.contains('\u2013') == False) & (df.index < 30)]

# th = th.loc[th['section']]
# df.index.rename("oid")



th = th.assign(section='summary')


th['order'] = ''
th['en'] = th['en'].str.split(':')
th = th.explode('en')
th_count = th.shape[0]
for i in range(0,th_count):
    th['order'].values[i] = i
    # th = th['order'].values[i] = i
# for row in th.itertuples():
#     th.at[row.Index,'Index'].assign(order=row.Index)

# th = th.set_index(['index','section']).apply(lambda x: x.str.split(':').explode())
# .reset_index()

# th = th.concat([Series(row['var2'], row['var1'].split(','))              
#                     for _, row in a.iterrows()]).reset_index()


# for i in range(1,th_count):
#     print(i)
# th.loc[:,'section'] = 'summary'
# th.loc[:,'order'] = 1
th = th.set_index('order',drop=False)
# th = th.reset_index(inplace=True)
# th
# th
# final_df = final_df.append(th).copy()
# final_df
th



final_df = final_df.append(th[:-1])
final_df

,section,order,line,altline,en,es,zh-TW,vi,ko,ja,hy,ru,schedule
34,header,1,NaN,NaN,Metro is making service changes.,Metro está haciendo cambios en el servicio.,Metro將對服務進行變更。,Metro sắp tiến hành thay đổi dịch vụ.,Metro는 버스편 운행 변경을 진행하고 있습니다.,Metroはサービスを変更しています。,Metro պոլիտենը փոփոխություններ է կատարում ծառա...,Metro вносит ряд изменений в расписание движения.,NaN
35,header,2,NaN,NaN,"New schedules start September 12, 2021.",Los nuevos horarios comienzan el 12 de septiem...,新安排將從2021年9月12日開始。,Lịch hoạt động mới sẽ bắt đầu có hiệu lực vào ...,신설 운행 일정은 2021년 9월 12일부터 시작합니다.,新しいスケジュールは2021年9月12日から開始されます。,Նոր ժամանակացույցերը կմեկնարկեն 2021 թվականի ս...,Новое расписание вводится в действие 12 сентяб...,NaN
0,summary,0,NaN,NaN,"Starting on Sunday, September 12, 2021, Metro ...","A partir del domingo, 12 de septiembre de 2021...",自2021年9月12日週日開始， Metro將對服務進行變更， 以便為您帶來更好的巴士 體驗。,"Bắt đầu từ Chủ Nhật, ngày 12 tháng 9 năm 2021,...","2021년 9월 12일, 일요일부터 개시, Metro가 향상된 버스 경험을 제공해 ...",2021年9月12日 日曜日より、 Metroは お客様により良いバスの旅をご体験いただける...,"2021 թվականի կիրակի, սեպտեմբերի 12-ից սկսած, M...","Начиная с субботы, 12 сентября 2021 года, Metr...",NaN
1,summary,1,NaN,NaN,We’re realigning routes to better match travel...,Estamos reorganizando las rutas para que se aj...,我們將重新調整行駛路線， 以讓其與乘車模式更加匹配， 並且會增加出行路線，以便實施 Next...,Chúng tôi sẽ điều chỉnh lại các tuyến lộ trình...,더 원할한 통행 패턴 반영을 위한 노선 재편과 운행 증편을 통해 NextGen 버스...,もっと旅行パターンに合わせたルートの再編成を行い、 次世代バスプラン (NextGen Bu...,Մենք փոփոխություններ ենք կատարում երթուղիներու...,"Мы изменяем схемы движения, чтобы привести их ...",NaN
2,summary,2,NaN,NaN,Some bus stops have been removed to improve tr...,Se eliminaron algunas paradas de autobús para ...,為縮短乘車時間已停用部分巴士站。,Một số trạm dừng xe buýt đã bị xóa bỏ để cải t...,운행 시간 개선을 위해 일부 버스 정류소를 폐쇄하였습니다.,いくつかのバス停は移動時間を改善するために撤去されました。,Որոշ ավտոբուսային կանգառներ հանվել են` ճանապար...,"Некоторые автобусные остановки отменены, чтобы...",NaN
3,summary,3,NaN,NaN,The following lines will have additional trips...,Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日、 週六及週日提供額外的出行服務： 2, 4, 16, 33, 45, 5...",Các tuyến sau đây sẽ có thêm chuyến vào các ng...,"다음 노선에는 주중, 토요일과 일요일에 운행을 증편할 예정입니다: 2, 4, 16,...","次の路線は、 平日、土曜日、日曜日の本数が追加されます: 2, 4, 16, 33, 45,...",Հետևյալ գծերը կունենան լրացուցիչ ուղևորություն...,По следующим маршрутам вводятся дополнительные...,NaN
4,summary,4,NaN,NaN,"2, 4, 16, 33, 45, 55, 70, 76, 92, 110, 117, 1...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日、 週六及週日提供額外的出行服務： 2, 4, 16, 33, 45, 5...",Các tuyến sau đây sẽ có thêm chuyến vào các ng...,"다음 노선에는 주중, 토요일과 일요일에 운행을 증편할 예정입니다: 2, 4, 16,...","次の路線は、 平日、土曜日、日曜日の本数が追加されます: 2, 4, 16, 33, 45,...",Հետևյալ գծերը կունենան լրացուցիչ ուղևորություն...,По следующим маршрутам вводятся дополнительные...,NaN
5,summary,5,NaN,NaN,"WEEKDAYS, the following lines will have additi...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日提供 額外的出行服務： 10, 14, 28, 40, 51, 53, 7...","VÀO CÁC NGÀY TRONG TUẦN, những tuyến sau sẽ đư...","주중에 다음 노선은 운행을 증편할 예정입니다: 10, 14, 28, 40, 51, ...","平日は、次の路線の本数が 追加されます: 10, 14, 28, 40, 51, 53, 7...","ԱՇԽԱՏԱՆՔԱՅԻՆ ՕՐԵՐԻՆ, հետևյալ գծերը կունենան լր...",По БУДНЯМ по следующим маршрутам будут осущест...,NaN
6,summary,6,NaN,NaN,"10, 14, 28, 40, 51, 53, 78/79, 81, 90, 108, 1...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在工作日提供 額外的出行服務： 10, 14, 28, 40, 51, 53, 7...","VÀO CÁC NGÀY TRONG TUẦN, những tuyến sau sẽ đư...","주중에 다음 노선은 운행을 증편할 예정입니다: 10, 14, 28, 40, 51, ...","平日は、次の路線の本数が 追加されます: 10, 14, 28, 40, 51, 53, 7...","ԱՇԽԱՏԱՆՔԱՅԻՆ ՕՐԵՐԻՆ, հետևյալ գծերը կունենան լր...",По БУДНЯМ по следующим маршрутам будут осущест...,NaN
7,summary,7,NaN,NaN,"SATURDAYS and SUNDAYS, the following lines hav...",Las siguientes líneas tendrán viajes adicional...,"以下路線將在週六及週日提供 額外的出行服務： 154, 155, 210","VÀO THỨ BẢY và CHỦ NHẬT, những tuyến sau sẽ đư...","토요일과 일요일에 다음 노선은 운행을 증편할 예정입니다: 154, 155, 210","土曜日、日曜日は、 次の路線の本数が追加されます: 154, 155, 210","ՇԱԲԱԹ և ԿԻՐԱԿԻ օրերին, հետևյալ գծե

In [ ]:
final_df.to_json('final_takeone.json')

### 2.2. Adding the `details`/lines section

First we will get all the lines in:

In [ ]:
# data = []
# lines = []
# keys = None
lines_df = pd.read_csv('data/mybus-sep-2021 - Lines.csv', index_col=0)
lines_takeone_df = df.loc[(df['en'].str.contains('\u2013')) & (df['en'].str.contains('B Line, D Line') == False)]

# lines_takeone_df = lines_takeone_df.index.rename("oid")
# lines_takeone_df = lines_takeone_df.set_index('order')

df.loc[(df['en'].str.contains('\u2013') == False) & (df.index < 30)]

lines_df['AltLine'] = lines_df.AltLine.fillna(0).astype(int)
all_lines = lines_df[['Line Label',"AltLine"]]
# all_lines

lines_count = all_lines.shape[0]

all_lines['order'] = ''
# all_lines['line'] = 

for i in range(0,lines_count):
    all_lines['order'].values[i] = i+1
all_lines.reset_index(inplace=True)
all_lines = all_lines.rename(columns={"Line Label":"line_label","Line Number":"line"})
# all_lines.index.rename('line', inplace=True)
all_lines

In [ ]:
line = lines_takeone_df.en.str.split('–')

lines_takeone_df['line'] = lines_takeone_df.en.str.split('–').str[0]

lines_takeone_df = lines_takeone_df.assign(var1=lines_takeone_df.line.str.split('/')).explode('var1')
# dupes = lines_takeone_df.loc[(lines_takeone_df.duplicated(subset=['line']))]

lines_takeone_df['line'] = lines_takeone_df.loc[(lines_takeone_df.duplicated(subset=['line']))]
# dupes = lines_takeone_df.duplicated(subset=['line'])
# dupes
# lines_takeone_df = lines_takeone_df.set_index('line')
# lines_takeone_df.tail()
# result = pd.merge(all_lines, lines_takeone_df, on=["line","line"])
lines_takeone_df
# result = pd.concat([all_lines,lines_takeone_df],axis=1)
# result = all_lines.join(lines_takeone_df,how="right")
# result = pd.merge(all_lines,lines_takeone_df,how="outer",on=["line","line"])

# lines_takeone_df.head()
# all_lines.head()
# lines_takeone_df.head(2)
# result

In [ ]:
# final_df.to_csv('finaltest.csv')
final_df.to_json('finaltest.json')

In [ ]:


df = pd.DataFrame(data)
df

finalTable

In [ ]:
# all_lines.set_index('Line Label')
# all_lines = lines_df

# all_lines = lines_df[[lines_df['Line Label'],lines_df['AltLine'].fillna(0).astype(int)]]
# lines_df = lines_df[['Line Label','AltLine']]
# lines_df